** Working with Missing Data **

**Utilizing the California Housing dataset**

# Setup

In [1]:
# Import package dependencies
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from ml_metrics import rmse
import matplotlib.pyplot as plt
from sklearn import datasets

# Loading Data

In [2]:
# Load in the dataset
california = datasets.fetch_california_housing()
print(california.data.shape)

(20640, 8)


In [3]:
print(california.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])


In [4]:
print(california.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [5]:
california.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [6]:
# Convert the matrix to pandas
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,4.526,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,3.585,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,3.521,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,3.413,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.422,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [7]:
cal.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,2.068558,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.153956,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.149990,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,1.196000,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,1.797000,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,2.647250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,5.000010,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


# Functions

In [76]:
def doModel(train_set,test_set,res_frame,name,imputation='none'):
    # Converting the training and testing datasets back to matrix-formats
    X_train = train_set.iloc[:, 1:].values # returns the data; excluding the target
    Y_train = train_set.iloc[:, 0].values # returns the target-only
    X_test = test_set.iloc[:, 1:].values # ""
    Y_test = test_set.iloc[:, 0].values # ""

    #model
    reg = LinearRegression(normalize=True).fit(X_train, Y_train)
    
    
    print(np.round(reg.coef_,2))
    #Metrics
    Y_pred = reg.predict(X_test)
    print(np.round(X_test[0:10],2))
    print(np.round(Y_test[0:10],2))
    mae = mean_absolute_error(Y_test,Y_pred)
    mse = mean_squared_error(Y_test,Y_pred)
    rmse_val = rmse(Y_test,Y_pred)
    r2 = r2_score(Y_test,Y_pred)
    #orignals
    if(res_frame.shape[1]==0):
        mae_diff=np.nan
        mse_diff=np.nan
        rmse_diff=np.nan
        r2_diff=np.nan
    else:
        mae_diff = mae-res_frame[res_frame['imputation']=='none']['mae']
        mse_diff = mse-res_frame[res_frame['imputation']=='none']['mse']
        rmse_diff = rmse_val-res_frame[res_frame['imputation']=='none']['rmse']
        r2_diff = r2-res_frame[res_frame['imputation']=='none']['R2']
    
    #return dataframe with metrics
    res_frame = res_frame.append(pd.DataFrame({'data':name,
                   'imputation':imputation,
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val, 
                   'R2':r2,
                   'mae_diff':mae_diff,
                   'mse_diff':mse_diff,
                   'rmse_diff':rmse_diff,
                   'R2_diff':r2_diff}, index=[0]))
    return(res_frame,reg)
    

In [13]:
def imputeBin(in_sample, out_sample,sel_col,impute_from):
    binned = pd.qcut(cal[impute_from],q=10)
    for b in binned.cat.categories:
        in_sample.loc[in_sample[impute_from].between(b.left,b.right),sel_col] = (
            in_sample[in_sample[impute_from].between(b.left,b.right)]
            .apply(lambda x:sample(list(out_sample[out_sample[impute_from].between(b.left,b.right)][sel_col]),1)[0],axis=1)
        )
        
    imputed_data = pd.concat([in_sample, out_sample])
    imputed_data = imputed_data.sort_index()
    return(imputed_data)
        #in_sample[~in_sample['HouseAg

In [60]:
def imputeMedian(in_sample, out_sample,sel_col,impute_from):
    in_sample[sel_col] = in_sample[sel_col].fillna(out_sample[sel_col].median())
    #in_sample[sel_col] = 1
    
    imputed_data = pd.concat([in_sample, out_sample])
    imputed_data = imputed_data.sort_index()
    return(imputed_data)
        #in_sample[~in_sample['HouseAg

# Baseline Model

In [77]:
# Create training and testing sets (cross-validation not needed)
train_set = cal.sample(frac=0.7, random_state=100)
test_set = cal[~cal.isin(train_set)].dropna()

# Get the training and testing row indices for later use
train_index = train_set.index.values.astype(int)
test_index = test_set.index.values.astype(int)

res_frame_baseline, reg = doModel(train_set,test_set,res_frame=pd.DataFrame(),name='original',imputation='none')
display(res_frame_baseline)  

[ 0.01  0.36 -1.38 -0.   -0.   -0.73 -0.72]
[[ 5.2000e+01  4.7600e+00  1.1000e+00  4.1300e+02  2.1400e+00  3.7850e+01
  -1.2225e+02]
 [ 5.2000e+01  4.9700e+00  9.9000e-01  1.5510e+03  2.1700e+00  3.7840e+01
  -1.2225e+02]
 [ 5.2000e+01  4.0500e+00  9.7000e-01  6.4800e+02  2.1400e+00  3.7850e+01
  -1.2227e+02]
 [ 4.0000e+01  4.5200e+00  1.1100e+00  4.0900e+02  2.4600e+00  3.7850e+01
  -1.2227e+02]
 [ 5.2000e+01  4.7800e+00  1.0600e+00  1.1020e+03  2.7800e+00  3.7850e+01
  -1.2228e+02]
 [ 5.0000e+01  4.4000e+00  1.0400e+00  1.1310e+03  2.3900e+00  3.7840e+01
  -1.2228e+02]
 [ 5.2000e+01  3.4900e+00  1.0300e+00  1.3770e+03  1.9800e+00  3.7830e+01
  -1.2226e+02]
 [ 5.2000e+01  4.2200e+00  1.0600e+00  9.4600e+02  2.3900e+00  3.7830e+01
  -1.2226e+02]
 [ 5.2000e+01  4.9400e+00  1.0700e+00  6.6000e+02  2.2600e+00  3.7830e+01
  -1.2226e+02]
 [ 5.2000e+01  4.3400e+00  1.1000e+00  7.1800e+02  1.8800e+00  3.7830e+01
  -1.2226e+02]]
[2.7  2.61 1.56 1.48 1.06 1.09 1.76 1.55 1.84 1.82]


,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,0.678033,0.80827,0.899039,0.390038,NaN,NaN,NaN,NaN


In [27]:

# Find the variable with the largest "normalized" coefficient value
print('The positive(max) coef-value is {}'.format(max(reg.coef_))) # Positive Max
#print('The abs(max) coef-value is {}'.format(max(reg.coef_, key=abs))) # ABS Max
#max_var = max(reg.coef_) # Positive Max
max_var = max(reg.coef_, key=abs) # ABS Max
var_index = reg.coef_.tolist().index(max_var)
print('The variable associated with this coef-value is {}'.format(california.feature_names[var_index]))

reg.coef_

The positive(max) coef-value is 0.3568699601528548
The variable associated with this coef-value is AveRooms


array([ 2.22212494e-03,  3.56869960e-01, -1.35904527e+00, -2.39259058e-05,
       -1.08871038e-03, -7.44439333e-01, -7.39272800e-01])

## Sel Object Column 

In [63]:
sel_col = 'AveBedrms'

## Correlations

In [18]:
train_set.corr()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
MedInc,1.000000,0.108943,0.170814,-0.061674,-0.028249,-0.021160,-0.149134,-0.042163
HouseAge,0.108943,1.000000,-0.173721,-0.095357,-0.305529,0.016458,0.013948,-0.109523
AveRooms,0.170814,-0.173721,1.000000,0.795184,-0.077868,-0.004450,0.111364,-0.027392
AveBedrms,-0.061674,-0.095357,0.795184,1.000000,-0.072633,-0.005975,0.073722,0.019007
Population,-0.028249,-0.305529,-0.077868,-0.072633,1.000000,0.080436,-0.108946,0.101335
AveOccup,-0.021160,0.016458,-0.004450,-0.005975,0.080436,1.000000,0.007245,-0.001679
Latitude,-0.149134,0.013948,0.111364,0.073722,-0.108946,0.007245,1.000000,-0.923624
Longitude,-0.042163,-0.109523,-0.027392,0.019007,0.101335,-0.001679,-0.923624,1.000000


# Select 1%, 5% 10%, 20%, 33%, and 50% of your data in a single column

In [78]:
samples = [.01,.05,.1,.2,.33,.5]
from random import sample 
impute_from = 'AveBedrms'
res_out = pd.DataFrame()
res_frame = res_frame_baseline
print(california.feature_names)
for s in samples:
    imputation = 'impute ' + str(int(s*100)) + '%'
    print(imputation)
    in_sample = cal.sample(frac=s, random_state=int(1701*s))
    out_sample = cal[~cal.isin(in_sample)].dropna()
    in_sample[sel_col] = np.nan
    
    #imputation
    #imputed_data=imputeBin(in_sample, out_sample,sel_col=sel_col,impute_from=impute_from)
    imputed_data=imputeMedian(in_sample, out_sample,sel_col=sel_col,impute_from=impute_from)
    print(np.mean(imputed_data[sel_col]))
    #prep for model
    train_set = imputed_data.iloc[train_index]
    test_set = imputed_data.iloc[test_index]
    #do model
    res_frame, reg = doModel(train_set,test_set,res_frame=res_frame,name='original',imputation=imputation)

display(res_frame)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
impute 1%
1.0962059373092556
[ 0.01  0.36 -1.38 -0.   -0.   -0.73 -0.72]
[[ 5.2000e+01  4.7600e+00  1.1000e+00  4.1300e+02  2.1400e+00  3.7850e+01
  -1.2225e+02]
 [ 5.2000e+01  4.9700e+00  9.9000e-01  1.5510e+03  2.1700e+00  3.7840e+01
  -1.2225e+02]
 [ 5.2000e+01  4.0500e+00  9.7000e-01  6.4800e+02  2.1400e+00  3.7850e+01
  -1.2227e+02]
 [ 4.0000e+01  4.5200e+00  1.1100e+00  4.0900e+02  2.4600e+00  3.7850e+01
  -1.2227e+02]
 [ 5.2000e+01  4.7800e+00  1.0600e+00  1.1020e+03  2.7800e+00  3.7850e+01
  -1.2228e+02]
 [ 5.0000e+01  4.4000e+00  1.0400e+00  1.1310e+03  2.3900e+00  3.7840e+01
  -1.2228e+02]
 [ 5.2000e+01  3.4900e+00  1.0300e+00  1.3770e+03  1.9800e+00  3.7830e+01
  -1.2226e+02]
 [ 5.2000e+01  4.2200e+00  1.0600e+00  9.4600e+02  2.3900e+00  3.7830e+01
  -1.2226e+02]
 [ 5.2000e+01  4.9400e+00  1.0700e+00  6.6000e+02  2.2600e+00  3.7830e+01
  -1.2226e+02]
 [ 5.2000e+01  4.3400e+00  

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,0.678033,0.808270,0.899039,0.390038,NaN,NaN,NaN,NaN
0,original,impute 1%,0.678994,0.813375,0.901873,0.386185,0.000961,0.005105,0.002835,-0.003853
0,original,impute 5%,0.687590,1.148859,1.071849,0.133012,0.009557,0.340589,0.172810,-0.257026
0,original,impute 10%,0.685102,1.068559,1.033711,0.193611,0.007069,0.260288,0.134672,-0.196427
0,original,impute 20%,0.690464,1.011840,1.005903,0.236414,0.012432,0.203570,0.106864,-0.153624
0,original,impute 33%,0.701821,1.123345,1.059880,0.152266,0.023788,0.315075,0.160841,-0.237771
0,original,impute 50%,0.694385,0.855328,0.924839,0.354526,0.016352,0.047058,0.025801,-0.035512
